In [1]:
import os
from datetime import datetime
from pathlib import Path

# import h5py
import scipy
import json
import pickle
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Duplicate key in file PosixPath('/home/lxh/miniconda3/envs/rul_prediction/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 268 ('font.family         : sans-serif        ')
Duplicate key in file PosixPath('/home/lxh/miniconda3/envs/rul_prediction/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 272 ('font.sans-serif: DejaVu Sans, Bitstream Vera Sans, Computer Modern Sans Serif, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif')


In [2]:
%matplotlib inline
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

In [4]:
def load_mat(file_path):
    filename = os.path.basename(file_path).split(".")[0] # eg.B0005
    data = scipy.io.loadmat(file_path)
    # print(type(data)) # dict
    # print(data.keys()) # dict_keys(['__header__', '__version__', '__globals__', 'B0005'])
    # print(data["B0005"][0][0][0][0].shape) # (616,)，实验持续了616个循环
    # print(type(data["B0005"][0][0][0][0][0])) # 用scipy读取mat文件，读取到结构体返回 numpy.void 类型数据

    cols = data[filename][0][0][0][0]
    
    data = []
    
    for col in cols:
        _ = {}
        parameter = {}
        keys = list(col[3][0][0].dtype.fields.keys())
        if str(col[0][0]) != "impedance":
            for idx, key in enumerate(keys):
                parameter[key] = list(col[3][0][0][idx][0])

        operation_type = str(col[0][0])
        temperature = str(col[1][0][0])
        time = str(datetime(
            int(col[2][0][0]), int(col[2][0][1]), int(col[2][0][2]), int(col[2][0][3]), int(col[2][0][4]), int(col[2][0][5])
        ))
        _["type"], _["temperature"], _["time"], _["data"] = operation_type, temperature, time, parameter
        data.append(_)

    return data

In [12]:
def data2csv(input, path):
    """ 数据保存到csv文件中 """
    features, label = input

    with open(path, "w", encoding="utf-8") as f:
        writer = csv.writer(f)
        for i in range(len(label)):
            row = list(features[i])
            row.append(label[i].item())
            print(row)
            break
            writer.writerow(row)

def save_data(data, path):
    features, label = data
    assert len(features) == len(label), f"输入特征{len(features)}和样本形状不一致{len(label)}"
    with open(path, "w", encoding="utf-8") as f:
        writer = csv.writer(f)
        for i in range(len(features)):
            row = list(features[i])
            row.append(label[i])
            print(row)
            writer.writerow(row)

In [6]:
def extract_charge(data):
    charge_input = []
    bcycle = len(data) # 616
    print(bcycle)
    # 最后一组数据的测量点过少，舍弃
    for i in range(0, bcycle - 1):
        if data[i]["type"] == "charge":
            end = len(data[i]["data"]["Voltage_measured"])
            le = end % 10

            vTemp = np.array(data[i]["data"]["Voltage_measured"][0:end-le])
            vTemp = vTemp.reshape(int(len(vTemp)/10), -1, order="F")
            print("1: ", vTemp.shape)
            print(vTemp)
            vTemp = vTemp.mean(axis=0)
            print("2: ", vTemp.shape)
            print(vTemp)

            iTemp = np.array(data[i]["data"]["Current_measured"][0:end-le])
            iTemp = iTemp.reshape(int(len(iTemp)/10), -1, order="F")
            iTemp = iTemp.mean(axis=0)

            tTemp = np.array(data[i]["data"]["Temperature_measured"][0:end-le])
            tTemp = tTemp.reshape(int(len(tTemp)/10), -1, order="F")
            tTemp = tTemp.mean(axis=0)

            charge_input.append(np.hstack((vTemp, iTemp, tTemp)))

    charge_input = np.array(charge_input)
    # print(charge_input.shape)
    return charge_input

def extract_discharge(data):
    bcycle = len(data)
    capacity = []
    for i in range(0, bcycle):
        if data[i]["type"] == "discharge":
            capacity.append(data[i]["data"]["Capacity"][0])
    return capacity

def max_min_norm(charge_input, init, cap):
    print(charge_input.shape)
    
    r = np.max(charge_input, axis=0) - np.min(charge_input, axis=0)
    print("here")
    print(np.max(charge_input, axis=0))
    print(np.min(charge_input, axis=0))
    print(r)
    print("here")
    X = (charge_input - np.min(charge_input, axis=0)) / r

    comp = len(charge_input) - len(cap)
    y = [init for _ in range(comp)] + cap
    ym = np.min(y)
    yr = np.max(y) - np.min(y)
    y = (y - ym) / yr
    return X, y, ym, yr

def min_max_scaler(data):
    min, max = np.min(data), np.max(data)
    r = max - min
    data = (data - min) / r
    return  data, min, r

def de_min_max_scaler(data, min, r):
    return data * r + min

In [14]:
battery_names = ["B0005", "B0006", "B0007", "B0018"]
# battery_name = ["B0018"]
battery_init_capacity = [1.86, 2.04, 1.89, 1.86]
# data_root_path = "/home/lxh/Desktop/battery_datasets/nasa/5_Battery_Data_Set/1_BatteryAgingARC-FY08Q4"
data_root_path = "./nasa_battery_aging_dataset/"
data_path = [os.path.join(data_root_path, name + ".mat") for name in battery_names]

nasa_data = {}
# capacity = {}
for name, path in zip(battery_names, data_path):
    nasa_data[name] = load_mat(path)

denorm = []
for index, cell in enumerate(list(nasa_data.keys())):
    charge_count, dischareg_count = 0, 0
    print(f"cell {cell}:")
    init_capacity = battery_init_capacity[index]
    cell_data = nasa_data[cell]
    cell_data_len = len(cell_data)
    print(f"\tlength of cell data: {cell_data_len}")
    for i in range(cell_data_len - 1):
        if cell_data[i]["type"] == "charge":
            charge_count += 1
        if cell_data[i]["type"] == "discharge":
            dischareg_count += 1
    print(f"\tcharge count: {charge_count}, discharge count: {dischareg_count}, len(charge series) >= len(discharge series): {charge_count > dischareg_count}")
    charge_input = extract_charge(cell_data)
    capacity = extract_discharge(cell_data)
    print(charge_input)
    X, y, ym, yr = max_min_norm(charge_input, battery_init_capacity[index], capacity)
    print(f"\t{cell} data loaded")
    csv_file_path = os.path.join(data_root_path, cell.lower() + ".csv")
    save_data((X, y), csv_file_path)
    print(f"\tdata save to file: {csv_file_path}")
    denorm.append((ym, yr))
    print("----------\n")

print(denorm)

# b0018 = nasa_data["B0018"]
# init_capacity_b0018 = battery_init_capacity[-1]
# # print(b0018[0])
# charge_input_b0018 = extract_charge(b0018)

# print(charge_input_b0018[0])
# capacity_b0018 = extract_discharge(b0018)
# X_b0018, Y_b0018, ym, yr = max_min_norm(charge_input_b0018, init_capacity_b0018, capacity_b0018)

# print(type(X_b0018), X_b0018.shape)
# print(type(Y_b0018), Y_b0018.shape)
# print(X_b0018[0])
# save_data((X_b0018, Y_b0018), os.path.join(data_root_path, "b0018.csv"))
# data2csv((X_b0018, Y_b0018), os.path.join(data_root_path, "b0018.csv"))
# print(f"ym: {ym}, yr: {yr}")

cell B0005:
	length of cell data: 616
	charge count: 169, discharge count: 168, len(charge series) >= len(discharge series): True
616
1:  (78, 10)
[[3.87301722 4.13123541 4.1762124  4.20782153 4.20978139 4.2058221
  4.20646844 4.20603644 4.20639054 4.20571813]
 [3.47939356 4.13183215 4.17663518 4.20765403 4.20958169 4.20501265
  4.20642732 4.20630483 4.20600337 4.2057691 ]
 [4.00058782 4.13231609 4.1774318  4.20746647 4.20910017 4.20505675
  4.20664243 4.2064063  4.2062531  4.20579774]
 [4.01239519 4.1328077  4.17789255 4.20823532 4.20938388 4.20525484
  4.20581669 4.20649455 4.20641592 4.20573034]
 [4.01970806 4.13349746 4.17861842 4.20782893 4.20811495 4.20547255
  4.20493724 4.20662282 4.20623889 4.20578899]
 [4.02540947 4.13410548 4.17919249 4.20763136 4.20823452 4.20598177
  4.20514409 4.2058586  4.20646365 4.20553262]
 [4.03063627 4.13446612 4.17974086 4.20709431 4.20850932 4.20522055
  4.20506395 4.20559323 4.20661728 4.20579817]
 [4.03534896 4.13522207 4.18061307 4.20755004 4.2

In [14]:
# 绘制B0005电池的容量退化曲线
# fig, ax = plt.subplots(1, figsize=(12, 8))
# plt.xlim(0, 180)
# plt.ylim(1.1, 2.1)
# plt.axhline(y=1.4, c="k", ls="--", label="失效阈值")
# ax.plot(capacity["B0005"][1], label="B0005电池", color="red")
# # ax.plot(capacity["B0006"][1], label="B0006")
# # ax.plot(capacity["B0007"][1], label="B0007")
# # ax.plot(capacity["B0018"][1], label="B00018")
# ax.legend()
# # plt.grid(b=None, which='major', axis='both')
# # ax.set(xlabel="Cycle", ylabel="Capacity(Ah)", title="Capacity Degradation in Cycle")
# ax.set(xlabel="充放电循环周期", ylabel="容量（Ah）")

# pretty_print = json.dumps(nasa_data['B0005'][:5], indent=4)
# print(pretty_print)


# 比较新老电池的VIT曲线
# data_path = "/home/lxh/Desktop/battery_datasets/nasa/5_Battery_Data_Set/1_BatteryAgingARC-FY08Q4/B0005.mat"
# data = load_mat(data_path)

# fresh_cell_time = [item / 60 for item in data[0]["data"]["Time"]]
# fresh_cell_v = data[0]["data"]["Voltage_measured"] # voltage
# fresh_cell_i = data[0]["data"]["Current_measured"] # current
# fresh_cell_t = data[0]["data"]["Temperature_measured"] # temperature


# old_cell_time = [item / 60 for item in data[167]["data"]["Time"]]
# old_cell_v = data[167]["data"]["Voltage_measured"] # voltage
# old_cell_i = data[167]["data"]["Current_measured"] # current
# old_cell_t = data[167]["data"]["Temperature_measured"] # temperature

# ax1 = plt.subplot(3, 1, 1)
# plt.title("fresh cell vs. old cell: voltage")
# plt.xlim(0, 180)
# plt.ylim(3.5, 4.5)
# ax1.set_xlabel("time(minute)")
# ax1.set_ylabel("voltage(V)")
# ax1.plot(fresh_cell_time, fresh_cell_v, label="fresh cycle (1st cycle)")
# ax1.plot(old_cell_time, old_cell_v, "--", label="old cycle (168th cycle)")
# ax1.legend()

# ax2 = plt.subplot(3, 1, 2)
# plt.title("fresh cell vs. old cell: current")
# plt.xlim(0, 180)
# plt.ylim(0, 2)
# ax2.set_xlabel("time(minute)")
# ax2.set_ylabel("current(A)")
# ax2.plot(fresh_cell_time, fresh_cell_i, label="fresh cycle (1st cycle)")
# ax2.plot(old_cell_time, old_cell_i, "--", label="old cycle (168th cycle)")
# ax2.legend()

# ax3 = plt.subplot(3, 1, 3)
# plt.title("fresh cell vs. old cell: temperature")
# plt.xlim(0, 180)
# plt.ylim(22, 30)
# ax3.set_xlabel("time(minute)")
# ax3.set_ylabel("temperature($^\circ$C)")
# ax3.plot(fresh_cell_time, fresh_cell_t, label="fresh cycle (1st cycle)")
# ax3.plot(old_cell_time, old_cell_t, "--", label="old cycle (168th cycle)")
# ax3.legend()

# plt.tight_layout()
# plt.show()


# 统计不同mode记录条数
# cnt_ch, cnt_disch, cnt_im = 0, 0, 0
# for i in nasa_data['B0005']:
#     if i['type'] == 'charge':
#         cnt_ch += 1
#     elif i['type'] == 'discharge':
#         cnt_disch += 1
#     else:
#         cnt_im += 1

# print(cnt_ch, cnt_disch, cnt_im, cnt_ch + cnt_disch + cnt_im)